In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import wget

quotes=[]  # a list to store quotes
for page in range(35, 10880):   
	URL = f"http://www.hypotheticalzeolites.net/dataconfirmed?page={page}&ent_pp=200&orderby=bgbener&toggle=-asc"
	r = requests.get(URL)
	   
	soup = BeautifulSoup(r.content, 'html5lib')	   
	table = soup.find('div', attrs = {'id':'navigation'}) 

	i = 0 
	for row in table.findAll('tr'):
		if i>1:
			j = 0
			quote = {}
			for col in row.findAll('td'):
				if j >0:
					cif_page_url = f"http://www.hypotheticalzeolites.net{row.a['href']}"
					quote['url'] = cif_page_url
					cif_r = requests.get(cif_page_url)
					   
					cif_soup = BeautifulSoup(cif_r.content, 'html5lib')	   
					cif_data = cif_soup.findAll('h2') 
					for h2 in cif_data:
						if h2.text == "CIF file":
							cif_file_url = f"http://www.hypotheticalzeolites.net{h2.a['href']}"
							quote['cif_file_url'] = cif_file_url
							wget.download(cif_file_url)

					break
				else:
					quote['id'] = col.text
					j+=1
			
			quotes.append(quote)
		else:
			i+=1
filename = 'data.csv'
with open(filename, 'w', newline='') as f:
    w = csv.DictWriter(f,['id','url','cif_file_url'])
    w.writeheader()
    for quote in quotes:
        w.writerow(quote)